In [1]:
import os
import math
import nltk
nltk.download('stopwords')
import re
from nltk import WordPunctTokenizer
from nltk.stem.porter import *
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/giwe7005/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# preprocess textual input --> tokens

def tokenize(string_input):
     string_input = re.sub(r'\W+', ' ', string_input)
     string_input = string_input.lower()
     tk = WordPunctTokenizer()
     tokens = tk.tokenize(string_input)
     stop_words = set(stopwords.words('english'))
     stemmer = PorterStemmer()
     clean_tokens = []
     for token in tokens:
          if token.isalpha() and token not in stop_words:
               clean_tokens.append(stemmer.stem(token))
     return clean_tokens

print(tokenize("When did Alessandro Volta improve  and popularize the electrophorus???"))

['alessandro', 'volta', 'improv', 'popular', 'electrophoru']


In [3]:
# CREATE CORPUS - extract files and create a dictionary of each file and their contents 
# {key = document_id: value = tokenised contents of document}

import json

def create_corpus():
    corpus = dict()
    for file in os.listdir(path="group_project/text_data"):
            # take only files with .clean suffix
            if file.endswith(".clean"):
                f = open(os.path.join("group_project/text_data", file), encoding="latin-1")
                file_name = file
                # gives the contents of each document just in string form
                file_contents = f.read()
                # tokenize contents of files
                tokens = tokenize(file_contents)
                # assign tokenize contents to file name entry in corpus
                corpus[file_name] = tokens
    return corpus

# create corpus 
corpus = create_corpus()

print(len(corpus))

# print example of corpus entry
print(corpus["S08_set4_a8.txt.clean"])
print(corpus["S09_set4_a8.txt.clean"])
print(corpus["S08_set1_a1.txt.clean"])




150
['amedeo', 'avogadro', 'caricatur', 'amedeo', 'avogadro', 'lorenzo', 'romano', 'amedeo', 'carlo', 'avogadro', 'count', 'quaregna', 'cerreto', 'august', 'juli', 'italian', 'savant', 'note', 'contribut', 'theori', 'molar', 'molecular', 'weight', 'tribut', 'number', 'elementari', 'entiti', 'atom', 'molecul', 'ion', 'particl', 'one', 'mole', 'substanc', 'known', 'avogadro', 'number', 'amedeo', 'avogadro', 'born', 'turin', 'august', 'nobl', 'ancient', 'famili', 'piedmont', 'itali', 'graduat', 'ecclesiast', 'law', 'earli', 'age', 'began', 'practic', 'soon', 'dedic', 'studi', 'physic', 'mathemat', 'call', 'posit', 'philosophi', 'start', 'teach', 'liceo', 'high', 'school', 'vercelli', 'famili', 'properti', 'publish', 'articl', 'titl', 'essai', 'une', 'manièr', 'de', 'détermin', 'le', 'mass', 'rel', 'de', 'molécul', 'élémentair', 'de', 'corp', 'et', 'le', 'proport', 'selon', 'lesquel', 'ell', 'entrent', 'dan', 'ce', 'combinaison', 'contain', 'famou', 'avogadro', 'hypothesi', 'titl', 'famou'

In [4]:
# create vocabulary, i.e. unique set of tokens that occur across all documents in corpus

def create_vocabulary(corpus):
     vocabulary = set()
     for file in corpus:
          for token in corpus[file]:
               # add token to vocabulary set 
               vocabulary.add(token)
     return sorted(vocabulary) # must be in a particular order 

def is_clean_token(token):
    return re.match(r"^[a-zA-Z\-]+$", token) is not None

# preprocess textual input --> tokens

create_vocab = create_vocabulary(corpus) # create unique set of terms that occur across all documents, n = 42,729

vocabulary = [token for token in create_vocab if is_clean_token(token)] # clean out strange vocabulary items

print(vocabulary)
print(len(vocabulary)) # n =27471

['aaa', 'aafc', 'aag', 'aalto', 'aaltonen', 'aamulehti', 'aamuna', 'aan', 'aanwerp', 'aapa', 'aardappelet', 'aardvark', 'aaron', 'aato', 'ab', 'aba', 'ababa', 'abacaxi', 'abahani', 'abalon', 'abandon', 'abang', 'abangan', 'abaya', 'abba', 'abbado', 'abbandon', 'abbasid', 'abbevil', 'abbey', 'abbot', 'abbott', 'abbrevi', 'abc', 'abckiria', 'abcnew', 'abdel', 'abdic', 'abdomen', 'abdomin', 'abdopu', 'abdu', 'abduct', 'abduh', 'abdul', 'abdullah', 'abdur', 'abe', 'abecedario', 'abel', 'abelson', 'aber', 'aberdar', 'aberdeen', 'aberr', 'aberrantli', 'abgeordnetenhau', 'abhor', 'abidjan', 'abigail', 'abil', 'abisara', 'abitibibowat', 'abl', 'ablaz', 'ablest', 'abnorm', 'abnudi', 'aboard', 'abolish', 'abolit', 'abolitionist', 'aborigin', 'abort', 'abraham', 'abram', 'abras', 'abreast', 'abridg', 'abroad', 'abrogan', 'abrsm', 'abrupt', 'abruptli', 'absalon', 'abscess', 'absenc', 'absent', 'absinth', 'absolut', 'absorb', 'absorpt', 'abstemi', 'abstract', 'abstractionist', 'abstrus', 'absurd', 

In [ ]:
# create document frequency dictionary - number of documents a given token appears in
# currently takes 3m 35s to run

document_frequency = {token: 0 for token in vocabulary}

for token in vocabulary:
    for document in corpus:
     if token in corpus[document]:
          document_frequency[token] += 1

print(document_frequency)


{'aaa': 1, 'aafc': 1, 'aag': 1, 'aalto': 2, 'aaltonen': 1, 'aamulehti': 1, 'aamuna': 1, 'aan': 1, 'aanwerp': 1, 'aapa': 1, 'aardappelet': 1, 'aardvark': 1, 'aaron': 1, 'aato': 1, 'ab': 3, 'aba': 1, 'ababa': 1, 'abacaxi': 1, 'abahani': 1, 'abalon': 4, 'abandon': 32, 'abang': 1, 'abangan': 1, 'abaya': 1, 'abba': 1, 'abbado': 2, 'abbandon': 1, 'abbasid': 2, 'abbevil': 1, 'abbey': 11, 'abbot': 2, 'abbott': 4, 'abbrevi': 14, 'abc': 8, 'abckiria': 1, 'abcnew': 1, 'abdel': 2, 'abdic': 4, 'abdomen': 8, 'abdomin': 5, 'abdopu': 1, 'abdu': 1, 'abduct': 1, 'abduh': 1, 'abdul': 3, 'abdullah': 1, 'abdur': 1, 'abe': 1, 'abecedario': 1, 'abel': 2, 'abelson': 1, 'aber': 1, 'aberdar': 1, 'aberdeen': 1, 'aberr': 2, 'aberrantli': 1, 'abgeordnetenhau': 1, 'abhor': 1, 'abidjan': 1, 'abigail': 3, 'abil': 41, 'abisara': 1, 'abitibibowat': 1, 'abl': 68, 'ablaz': 1, 'ablest': 1, 'abnorm': 4, 'abnudi': 1, 'aboard': 4, 'abolish': 9, 'abolit': 5, 'abolitionist': 3, 'aborigin': 7, 'abort': 5, 'abraham': 15, 'abram'

In [6]:
# pre-compute idf scores using document frequency dictionary

def calculate_idf(term, corpus_dictionary):
     """calculate inverted document frequency"""
     idf = math.log(len(corpus_dictionary)/((document_frequency[term])+1)) # can replace w/frequency count 
     return idf


idf_scores = {}

for item in vocabulary:
     if item not in idf_scores:
          idf_scores[item] = calculate_idf(item, corpus)
          #print(idf_scores[item])

print(idf_scores)


{'aaa': 4.31748811353631, 'aafc': 4.31748811353631, 'aag': 4.31748811353631, 'aalto': 3.912023005428146, 'aaltonen': 4.31748811353631, 'aamulehti': 4.31748811353631, 'aamuna': 4.31748811353631, 'aan': 4.31748811353631, 'aanwerp': 4.31748811353631, 'aapa': 4.31748811353631, 'aardappelet': 4.31748811353631, 'aardvark': 4.31748811353631, 'aaron': 4.31748811353631, 'aato': 4.31748811353631, 'ab': 3.624340932976365, 'aba': 4.31748811353631, 'ababa': 4.31748811353631, 'abacaxi': 4.31748811353631, 'abahani': 4.31748811353631, 'abalon': 3.4011973816621555, 'abandon': 1.5141277326297755, 'abang': 4.31748811353631, 'abangan': 4.31748811353631, 'abaya': 4.31748811353631, 'abba': 4.31748811353631, 'abbado': 3.912023005428146, 'abbandon': 4.31748811353631, 'abbasid': 3.912023005428146, 'abbevil': 4.31748811353631, 'abbey': 2.5257286443082556, 'abbot': 3.912023005428146, 'abbott': 3.4011973816621555, 'abbrevi': 2.302585092994046, 'abc': 2.8134107167600364, 'abckiria': 4.31748811353631, 'abcnew': 4.3

In [7]:
# turn corpus into pandas dataframe
idf_df_scores = pd.DataFrame.from_dict(idf_scores, orient='index', columns=['IDF Score'])

print(idf_df_scores)

# accessing an idf score: idf_df_scores[term]

           IDF Score
aaa         4.317488
aafc        4.317488
aag         4.317488
aalto       3.912023
aaltonen    4.317488
...              ...
zwischen    3.912023
zwoll       4.317488
zygomat     3.912023
zygoptera   4.317488
zz          4.317488

[27374 rows x 1 columns]


In [8]:
from collections import Counter
# TERM FREQUENCY MATRIX - PER DOCUMENT
# create term-document-matrix - every document converted to a vector corresponding to frequency of each vocab item appearing in that document

def term_document_matrix(corpus, vocab):
       data = {}
       for doc_id, tokens in corpus.items():
              freqs = Counter(tokens)
              for term, count in freqs.items():
                     if term in vocab:
                            if term not in data:
                                   data[term] = {}
                            data[term][doc_id] = count
       df = pd.DataFrame.from_dict(data, orient='index').fillna(0).astype(int)  
       return df


term_document_matrix = term_document_matrix(corpus, vocabulary)

print(term_document_matrix)

           S08_set4_a8.txt.clean  S09_set4_a8.txt.clean  \
amedeo                         5                      5   
avogadro                      33                     33   
caricatur                      1                      1   
lorenzo                        1                      1   
romano                         1                      1   
...                          ...                    ...   
uniondal                       0                      0   
cve                            0                      0   
lha                            0                      0   
peanut                         0                      0   
pepperidg                      0                      0   

           S10_set4_a8.txt.clean  S10_set5_a5.txt.clean  \
amedeo                         3                      0   
avogadro                      30                      0   
caricatur                      0                      1   
lorenzo                        1                      0

In [9]:
# calculate tf values

def calculate_tf(term, document, term_document_matrix):
     """calculate term frequency"""
     tf = term_document_matrix[term][document]/len(document) # tf = vocab_index[term, document] / len(document)
     return tf


print(term_document_matrix['S09_set2_a10.txt.clean']["amedeo"])


0


In [21]:
doc_ids = [file for file in os.listdir(path="group_project/text_data") if file.endswith('.clean')]

#sparse_dtype = pd.SparseDtype("float64", fill_value=0.0)
tfidf_df = pd.DataFrame(0.0, index=vocabulary, columns=doc_ids) #dtype=sparse_dtype)

def doc_vectorise(doc_ids, corpus, vocabulary):
    for doc in doc_ids:
        counts = Counter(corpus[doc])
        for term, freq in counts.items():
            if term in vocabulary:
                # calculate tf_idf
                term_document_matrix.loc[term, doc] = (freq / len(corpus[doc])) * idf_scores[term] # or use another tf scheme
    return term_document_matrix

tf_idf_docs_vector = doc_vectorise(doc_ids, corpus, vocabulary)
tf_idf_docs_vector.to_csv("tfidf_output_FINAL.csv")


In [18]:
# process query

def process_query(query):
    query = re.sub("\W", " ", query)
    query = query.strip().lower()
    query = " ".join([word for word in query.split() if word not in stopwords.words('english')])
    return query

test_query = "Was Alessandro Volta a professor of chemistry?"
print(process_query(test_query))

split_query = np.unique(test_query.split())

print(split_query)



#def query_score(vocab_index, query):
  #  for word in np.unique(query.)

alessandro volta professor chemistry
['Alessandro' 'Volta' 'Was' 'a' 'chemistry?' 'of' 'professor']


In [ ]:
# COSINE SIMILARITY - method to compare the query with each document in the corpus

def calculate_cosine_similiarity(doc_vector_a, doc_vector_b):
     """calculate cosine similarity in order to compare query vector with document vector"""
     co_sim = (doc_vector_a * doc_vector_b)/(len(doc_vector_a)*len(doc_vector_b))
     return co_sim

In [27]:
# PROCESS USER QUERY

def process_query(user_query):
     """process user query - tokenize"""
     tokens = tokenize(user_query) # apply same tokeizer used on corpus data
     return tokens



def vectorise_query(user_query, corpus_vector): # corpus vector = tf_idf dataframe
     corpus_vector["query_vector"] = 0
     print(corpus_vector["query_vector"])
     tokenized_query = process_query(user_query)
     counts = Counter(tokenized_query)
     print(counts)
     query_length = len(tokenized_query)
     for term, freq in counts.items():
          if term in corpus_vector.index:
               # calculate tf_idf
               corpus_vector["query_vector"][term] = freq/query_length * idf_scores[term] # or use another tf scheme
     return corpus_vector["query_vector"]


# term_document_matrix.loc[term, doc] = (freq / len(corpus[doc])) * idf_scores[term] # or use another tf scheme

print(tf_idf_docs_vector.index)
print(vectorise_query("Was Alessandro Volta a professor of chemistry?", corpus_vector=tf_idf_docs_vector))

Index(['amedeo', 'avogadro', 'caricatur', 'lorenzo', 'romano', 'carlo',
       'count', 'quaregna', 'cerreto', 'august',
       ...
       'rochford', 'weilburg', 'polari', 'loch', 'grouper', 'uniondal', 'cve',
       'lha', 'peanut', 'pepperidg'],
      dtype='object', length=27374)
amedeo       0
avogadro     0
caricatur    0
lorenzo      0
romano       0
            ..
uniondal     0
cve          0
lha          0
peanut       0
pepperidg    0
Name: query_vector, Length: 27374, dtype: int64
Counter({'alessandro': 1, 'volta': 1, 'professor': 1, 'chemistri': 1})
amedeo       0.0
avogadro     0.0
caricatur    0.0
lorenzo      0.0
romano       0.0
            ... 
uniondal     0.0
cve          0.0
lha          0.0
peanut       0.0
pepperidg    0.0
Name: query_vector, Length: 27374, dtype: float64


/tmp/ipykernel_1419812/1774593390.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  corpus_vector["query_vector"][term] = freq/query_length * idf_scores[term] # or use another tf scheme


In [ ]:
# MAIN FUNCTION - takes user input, processes user input, calculates user query tf idf value, turns it into vector of some kind, compare with tf-idf values of corpus documents, return top 5 documenst
def main():
     corpus = create_corpus() # {key = document_id: value = tokenised contents of document}
     print("corpus created")
     vocabulary = create_vocabulary(corpus) # create unique set of terms that occur across all documents, n = 42,729
     print("vocabulary created")
     # idf_values = {term: calculate_idf(term, corpus) for term in vocabulary} # calculate and store one IDF value per vocab term
     print("idf_values created")
     #inverted_index = {term: [] for term in vocabulary}
    # document_norms = {}

     """
     for document, tokens in corpus.items():
          print("this loop")
          tf_counts = collections.Counter(tokens)
          document_len = len(tokens)
          norm_squared = 0.0 # initialise document norm
          for term, count in tf_counts.items():
               tf = count/document_len
               tf_idf = tf*idf_values[term]
               norm_squared = tf_idf**2
               inverted_index[term].append((document, tf_idf))
          document_norms[document] = math.sqrt(norm_squared)
     """

     print(tokenize("Did Horace Greenley lose in the presidential elections of 1872"))
     print(len(vocabulary))
     test1 = process_query("I got a lotsa apples", corpus)
     test2 = process_query("Did Lincoln sign the National Banking Act of 1863?", corpus)
     #corpus_vectors = document_vectorisation(corpus)
     print(test2)
